<img src="https://github.com/rmlarose/qcbq/blob/master/img/banner.png?raw=true">

# Tutorial 3: Superconducting Qubits, Noise, and Decoherence

**Authors:** Ryan LaRose and Justin Lane.

<img src="https://www.researchgate.net/profile/Graham_Smetham/publication/268052917/figure/download/fig14/AS:668771421286410@1536458940555/Zureks-cartoon-the-two-truths-quantum-and-classical.jpg">

Many different physical systems can realize quantum bits. Among them, superconducting qubits are a current top contendor. This notebook serves as a tutorial for superconducting qubits in particular, but many concepts apply to the general qubit. We look in detail at *energy dissipation* and *decoherence* (or, the quantum-to-classical transition).

## Learning goals

(1) Understand how microwave pulses can manipulate superconducting qubits.

(2) Be able to define T1 and T2 times, and set up an experiment to measure them.

(3) Visualize pulse sequences in `qiskit.pulse`.

(4) Perform data-fitting to measure T1 and T2 times on IBM quantum devices.

## Helpful background

* Data fitting in Python using `scipy.optimize.curve_fit`. See [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) for a reference.
* Quantum mechanics for understanding single qubit Hamiltonians/gates.

In [ ]:
"""Imports."""
import matplotlib.pyplot as plt
import numpy as np
import scipy as scp

from IPython.display import clear_output, display

import qiskit.pulse as pulse
import qiskit

In [ ]:
"""Plotting style."""
%matplotlib inline
plt.rcParams.update({"font.size": 16, "font.weight": "bold"})

## How to manipulate qubits with pulses

This section is fairly technical. Those with appropriate background (at least quantum mechanics) will gain a working understanding of how qubits are manipulated; those without such a background will likely not benefit from this section. For the latter group, you can skip to the next section with the following message...

**Big picture takeaway:** We perform quantum gates on superconducting qubits by sending microwave pulses through control lines.

### The Qubit Hamiltonian

*Note: This section gives a brief exposition. More details can be found in Section III of this [paper](https://arxiv.org/abs/1904.06560).*

<img src="https://github.com/rmlarose/qcbq/blob/master/img/LC_JC_circuit.png?raw=true">

To a physicist a qubit is any two level quantum system, or two levels of a higher dimensional quantum system that we can isolate. For example, we could make a qubit out of a spin-1/2 particle in a magnetic field, with $|0\rangle$ being the spin down configuration and $|1\rangle$ being the spin up configuration. We could also make a qubit out of the $n=0$ (ground) and $n=1$ (first excited) states of a quantum harmonic oscillator (provided we have a way of isolating those two states from the rest of the harmonic oscillator ladder!)

Superconducting qubits take the second route: physicsts and engineers use **Josephson junctions**, a type of superconducting element that acts like a nonlinear inductor, to create **anharmonic oscillators** to use as qubits. When quantized, a regular LC oscillator (left) is simply a harmonic oscillator, with equal energy level spacings ($\hbar\omega_{01} = \hbar\omega_{12} =~...$). In this configuration, even if we only want to work with the $n=0$ and $n=1$ states, there is nothing stopping us from accidentally exciting the oscillator into the $n=2,3,4...$ states. However, when we replace the inductor with a josephson junction (right) we get an anharmonic oscillator with $\hbar\omega_{01} \neq \hbar\omega_{12}$. The unevenly spaced energy levels allow confine ourselves to the the lowest two levels (the **computational subspace**) of the harmonic oscillator ladder, and call those two levels our qubit.

The general Hamiltonian for a qubit in a time dependent electric field may be written as

\begin{equation}
    H = - \frac{\hbar\omega_q}{2} \sigma_{z} + \mathbf{d}\cdot\mathbf{E}(t) \sigma_{x}  = - \frac{\hbar\omega_q}{2} \sigma_{z} + \hbar\Omega\epsilon(t) \sigma_{x}
\end{equation}

Here, $\omega_q$ is the **qubit frequency** (anharmonic oscillator $\omega_{01}$ in our case), $\mathbf{E}$ is the electric field, and $\mathbf{d}$ is the "dipole moment" of the qubit (a measure of how the qubit couples to applied electric fields). The qubit frequency $\omega_q$ and dipole moment $\mathbf{d}$ are determined by parameters of the circuit (e.g., capicatance and inductance). In the second equation, we've combined $\mathbf{E}$ and $\mathbf{d}$ into a **drive strength** $\Omega$ and a unitless electric field $\epsilon(t)$.  As usual, the $\sigma$'s are Paulis

\begin{equation}
    \sigma_{x} := \left[ \begin{matrix}
    0 & 1 \\
    1 & 0 \\
    \end{matrix} \right]
    \qquad
    \text{and}
    \qquad
    \sigma_{y} := \left[ \begin{matrix}
    0 & -i \\
    i & 0 \\
    \end{matrix} \right]
    \qquad
    \text{and}
    \qquad
    \sigma_{z} := \left[ \begin{matrix}
    1 & 0 \\
    0 & -1 \\
    \end{matrix} \right] .
\end{equation}

### The drive Hamiltonian

We generally design our qubits to have frequencies $\omega_q$ in the several gigahertz range: this is because we manipulate qubits using nearly resonant electromagnetic fields, and microwave electronics have been extensively developed by the telecom industry.

In this spirit, we'll write our unitless electric field as an *envelope function* $s(t)$ multiplied by a sinusoid oscillating at some drive frequency $\omega_d$:

$$\epsilon(t) = s(t)\cos(\omega_dt + \phi)$$ 

Which makes our entire Hamiltonian

$$ H =  - \frac{\hbar\omega_q}{2} \sigma_{z} + \hbar\Omega s(t)\cos(\omega_dt + \phi)\sigma_{x}$$

It is now convenient to transform this Hamiltonian into the frame *rotating at the drive frequency* $\omega_q$, which simplifies the dynamics of the Hamiltonian$^1$. In this frame (and in the [rotating wave approximation](https://en.wikipedia.org/wiki/Rotating_wave_approximation)), the Hamiltonian takes the form 

\begin{equation}
    \tilde{H} = -\frac{\hbar}{2}\left[ \begin{matrix}
    \delta\omega & \Omega s(t)e^{-i\phi} \\
    \Omega s(t)e^{i\phi} & -\delta\omega \\
    \end{matrix} \right]
\end{equation}

where $\delta\omega = \omega_q - \omega_d$ is the qubit/drive **detuning**.

The Hamiltonian above can be split into two terms $\tilde{H} = \tilde{H}_q + \tilde{H}_d$


where 

\begin{equation}
    \tilde{H}_q = -\frac{\hbar}{2}\left[ \begin{matrix}
        \delta\omega & 0 \\
        0 & -\delta\omega \\
        \end{matrix} \right] \qquad \textbf{(qubit Hamiltonian)}
\end{equation}

and

\begin{equation}
    \tilde{H}_d = -\frac{\hbar \Omega s(t)}{2}\left[ \begin{matrix}
    0 & e^{-i\phi} \\
    e^{i\phi} & 0 \\
    \end{matrix} \right] \qquad \textbf{(drive Hamiltonian)}
\end{equation}

For the term $e^{i \phi} = cos \phi + i \sin \phi$, we often define

\begin{align}
I &:= \cos (\phi) \qquad \text{(the "in-phase" component)} \\
Q &:= \sin (\phi) \qquad \text{(the "out-of-phase" or "quadrature" component)}
\end{align}

for convenience.

### Implementing single qubit gates

**Question:** Show that if we apply a pulse at the qubit frequency, i.e., $\delta\omega = 0$, then

\begin{equation}
    \tilde{H} = \tilde{H}_d = - \frac{\hbar\Omega}{2}s(t)( I \sigma_x + Q \sigma_y) .
\end{equation}


<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

Thus, for a pulse at the qubit frequency, an *in-phase pulse* ($\phi = 0 \implies Q = 0$) corresponds to rotating the qubit about the $x$-axis. Similarly, an *out-of-phase pulse* ($\phi = \pi / 2 \implies I = 0$) corresponds to rotating the qubit about the $y$-axis. 

How long should we apply the pulse for? Solving Schrodinger's equation with, for example, the in-phase pulse above, we have

\begin{equation}
    U(t) = \exp \left ( \left[ \frac{i}{2} \Omega V_0 \int_{0}^{t} s(t') \, dt' \right] \sigma_x \right) 
\end{equation}

One can see this is the standard form of an $x$-rotation with angle

\begin{equation}
    \Theta(t) := - \Omega V_0 \int_{0}^{t} s(t') \, dt'
\end{equation}

If we want to rotate by, say, $\pi / 2$ radians about the $x$-axis, we thus solve the equation $\Theta(t) = \pi / 2$ for time $t$. This then tells us how long to apply the pulse. 

**Question**: Suppose your adviser came into the lab and demanded that all your qubit pulses have an envelope function of the form $s(t) = - t$. How long should you apply an in-phase pulse to implement a NOT gate: i.e., a rotation about the $x$-axis by $\pi$ radians. Express your answer in terms of $\Omega$. 

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

## Qiskit Pulse

Qiskit has a module called `Pulse` for creating pulse sequences and manipulating qubits using the techniques discussed above. This is nice functionality because it is at a lower level than the standard gates provided in Qiskit (e.g., $H$, $X$, $Y$, etc.). Unfortunately, we could not get access to `Pulse` for the bootcamp... but we did get access to real experimental data obtained using `Pulse`. We will use this to calibrate and benchmark the superconducting qubits on IBM Q devices, and we will learn the techniques to do so. 


Although we cannot currently use `Pulse`, the next cells show *how* we would use it (or, at least, set it up) if we had access to the `ibmq-poughkeepsie` device. More details on how to construct pulse sequences and run them on the Poughkeepsie device can be found [in this notebook](https://community.qiskit.org/textbook/ch-quantum-hardware/calibrating-qubits-openpulse.html).

In [ ]:
"""Load account: Only do this once."""
qiskit.IBMQ.load_account()

In [ ]:
"""Get a provider and see available backends."""
provider = qiskit.IBMQ.get_provider(hub="ibm-q")
print("Available backends:", *provider.backends())

In [ ]:
"""Select ibmq-poughkeepsie device to use Pulse."""
# backend = provider.get_backend("ibmq_poughkeepsie")
# config = backend.configuration()
# defaults = backend.defaults()

In [ ]:
"""Get a pulse channel system."""
# system = pulse.PulseChannelSpec.from_backend(backend)

Once you had `Pulse` imported and setup (if you had access), you could use it to apply precise pulse sequences to qubits. Although we cannot apply the pulses ourselves, other people can -- and have! In what follows, we will analyze multiple experiments at the "pulse level" and use data fitting techniques to determine qubit frequency, T1 time, and T2 time (to be defined below).

## Benchmarking qubits: Measuring frequency, T1, and T2 times

Now that we understand qubits and pulses, we can begin benchmarking qubits. The properties we will investigate are **energy dissipation** and **decoherence**.

## Energy dissipation and T1

<img src="https://github.com/rmlarose/qcbq/blob/master/img/t1.png?raw=true">

The state $|0\rangle$ is the ground state of the qubit, and the state $|1\rangle$ is the excited state, meaning the qubit has more energy in this state. Due to environmental interactions (e.g., exchanging energy with a photon), a qubit in the $|1\rangle$ state will lose energy and eventually "decay" into the ground state. Experimentally, this decay follows an exponential curve with exponent proportional to $1 / T_1$, where $T_1$ has units of time.

Explicitly, the probability of finding an excited qubit in the ground state after time $t$ is found to be

$$ p_0(t) = e^{ - t / T_1 } $$ 

### How to measure T1

The definition of T1 suggests a simple way to measure it.

1. Put the qubit in the $|1\rangle$ (excited) state.
1. Wait a time $t$.
1. Measure the qubit.

This is exactly how T1 times are measured in the lab. The process is illustrated in the above cartoon of the [Bloch sphere](https://en.wikipedia.org/wiki/Bloch_sphere), a graphical means of representing the state of a single qubit. In the far left, we start in the ground state $|0\rangle$. In the middle, we have put the qubit in the excited state $|1\rangle$. After waiting some time $t$, the qubit loses energy to its environment and is in a mixed state $\rho = p_0 |0\rangle\langle0| + p_1 |1\rangle\langle1|$, represented by a "vertically squished" Bloch sphere.

In [ ]:
"""Video showing conceptual visualization of the above experiment on the Bloch sphere."""
from IPython.display import YouTubeVideo
YouTubeVideo("g10PIHQ65L4", width=640, height=360)

### Pulse sequence

The pulse sequence for measuring T1 simply translates the above process. We apply an in-phase $\pi$-pulse at the qubit frequency to excite it from $|0\rangle$ to $|1\rangle$. Then we wait for a time $t$ and finally apply a measurement pulse. The entire pulse sequence is shown in the diagram below, made using Qiskit Pulse.

**Question:** Could we also use an *out-of-phase* (instead of *in-phase*) $\pi$-pulse at the qubit frequency in the above pulse sequence? Why or why not?

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

### TODO: Put pulse sequence image here.

### Raw data

This experiment was performed using `Pulse` with the above pulse schedule. The raw data can be read by executing the following cell.

In [ ]:
"""Load in the raw data."""
# TODO: Replace with real data.
T1real = 50  # microseconds
sigma = 5.0
times = np.linspace(0, 100, 1000)
data = np.exp(- times / T1real) + np.random.randn(len(times)) / sigma**2

The data file is formatted as follows. The array `times` contains the "wait time" (step 2 above) in microseconds, and the `measurements` array contains the number of times the excited state was measured. The total number of measurements for each wait time was 1000.

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, make a plot of the raw data obtained from the experiment. Your plot should have time on the horizontal axis and probability (frequency) of measuring the excited state on the vertical axis. Axis labels and a title never hurt anyone!

In [ ]:
"""Plot the raw data."""
### Your code here!
plt.figure(figsize=(12, 5))
plt.plot(times, data, linewidth=2);
plt.xlabel("Time [microseconds]");
plt.ylabel("Probability of excited state");
plt.title("T1 measurement data")
plt.grid();
plt.show()

### Fitting

We will now fit the data to an exponential curve to calculate the $T_1$ time.

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, define a function with `time` as the first argument and `T1` as the second argument, The function should return the exonential $e^{-t / T_1}$. 

In [ ]:
"""Do the data fitting."""
def fit(time: float, T1: float) -> float:
    return np.exp(-time / T1)  # Your code here!

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, use `scipy.optimize.curve_fit` or your favorite data-fitting tool to fit the exponential to the raw data.

In [ ]:
"""Use scipy.optimize.curve_fit to fit the data."""
optimal_params, covariance = scp.optimize.curve_fit(fit, times, data)
print(f"Computed T1 value: {round(optimal_params[0], 3)} microseconds.")

**Question:** What is your computed $T_1$ time? (Remember units!)

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

To make sure we got a good fit, it's always a good idea to plot the fit over the data.

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, plot the computed best-fit exponential curve and the raw data in the same plot.

In [ ]:
"""Plot your fit over the raw data here."""
# Compute the fit
fitvals = fit(times, *optimal_params)

# Plot the fit and data
plt.figure(figsize=(12, 5))
plt.plot(times, data, "-.", linewidth=2, label="Data");
plt.plot(times, fitvals, "-", linewidth=4, label="Fit");
plt.xlabel("Time [microseconds]");
plt.ylabel("Frequency of excited state");
plt.title("T1 measurement data");
plt.legend();
plt.grid();
plt.text(45, 0.8, f"T1 = {round(optimal_params[0], 3)} microseconds")
plt.show()

**Congrats!** You just measured the $T_1$ time of a superconducting qubit. Every few hours, the IBM Q devices are re-calibrated and fresh values of $T_1$ are put online. The method they use is the same method above.

**Question**: Search online for IBMQ devices. What are typical T1 values for other qubits and devices? (This [link](https://github.com/Qiskit/ibmq-device-information) may be useful.)

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

## Measure T2 time

When qubits interact with their environment, they lose more than just energy. **Coherence**, a defining feature of quantum mechanics and key departure from the classical world, is the ability for "particles" to exist in a superposition of (basis) states. For a single qubit, a coherent state can thus be written

\begin{equation}
    |\psi\rangle = \alpha |0\rangle + \beta |1\rangle .
\end{equation}

In the density matrix formalism, we may equivalently write

\begin{equation}
    \rho \equiv |\psi\rangle \langle \psi| = \left[ \begin{matrix} 
     |\alpha|^2 & \alpha \beta^* \\
     \alpha^* \beta & |\beta|^2 
    \end{matrix} \right]
\end{equation}

Note that the superposition is directly responsible for the off-diagonal terms in this matrix. Without superposition, diagonal elements are zero. (See *Technical note 1* below.)

**Question:** Show that the density matrix of $|\psi\rangle = |0\rangle$ has off-diagonal elements equal to zero. In particular, show that the density matrix is

\begin{equation}
    \rho \equiv |\psi\rangle \langle \psi| = \left[ \begin{matrix} 
     1 & 0 \\
     0 & 0
    \end{matrix} \right]
\end{equation}

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

Experimentally, coherence is found to decay exponentially. Such a loss of coherence due to environmental interactions is called **decoherence**. A model of the density matrix decohering can be written

\begin{equation}
    \rho (t) = \left[ \begin{matrix} 
     |\alpha|^2 & \alpha \beta^* e^{- t / T_2} \\
     \alpha^* \beta e^{-t / T_2} & |\beta|^2 
    \end{matrix} \right] .
\end{equation}

We will now do two experiments to measure the $T_2$ time of superconducting qubits. 

## Ramsey interferometry

One way to determine $T_2$ is to do a so-called Ramsey experiment or Ramsey pulse sequence.

This experiment starts in the ground state, then puts the qubit on the $y$-axis of the Bloch sphere. After a time $t$, the sphere contracts in the $xy$-plane (i.e., decoheres) due to interactions with the environment. A final $\pi/2$ pulse puts the qubit back on the $z$-axis in a mixed state $\rho = p_0 |0\rangle \langle0| + p_1 |1\rangle \langle 1|$, then the qubit is measured.

During step 2 (the "waiting" step), the Hamiltonian of the system is, from above, 

\begin{equation}
    \tilde{H} = -\frac{\hbar}{2}\left[ \begin{matrix}
    \delta\omega & 0 \\
    0 & -\delta\omega \\
    \end{matrix} \right] = -\frac{\hbar\delta\omega}{2}\sigma_z
\end{equation}

If the qubit is slightly detuned from the drive frequency (i.e., $\delta \omega = 0$), this is equivalent to applying a constant rotation about the $z$-axis of the Bloch sphere. The qubit state vector, in addition to decohering, will also slowly precess around the Bloch sphere at angular frequency $\delta\omega$, which will cause our decoherence data to have a sinusoidal component superimposed on it. We may extract $\delta\omega$ (and therefore $\omega_q$) from the frequency of this sinusoidal component. 

The video below shows the above description on the Bloch sphere.

In [ ]:
"""Video showing conceptual visualization of the above experiment on the Bloch sphere."""
from IPython.display import YouTubeVideo
YouTubeVideo("9Ekep8zgZHc", width=640, height=360)

### Pulse sequence

The Ramsey pulse sequence for determining $T_2$ is a translation of the above experimental description into pulses:

1. Apply a $\pi/2$ pulse about the $x$-axis to create a coherent state.
1. Wait for a time $t$.
1. Apply a $\pi/2$ pulse about the $x$-axis.

### TODO: Add image of qiskit pulse sequence here

### Raw data

Again, although we cannot use `Pulse` to perform this experiment, we have the next best thing: raw data. Execute the cell below to read in this data.

In [ ]:
"""Load in the raw data."""
# TODO: Replace with real data.
T2 = 80
domega = np.pi / 2
sigma = 4.0
times = np.linspace(0, 100, 500)  # microseconds
data = 0.5 + 0.5 * np.cos(domega * times) * np.exp(- times / T2) + np.random.randn(len(times)) / sigma**2

The format of this data is as follows. The `times` array contains the "wait times" in microseconds, and the `measurements` array contains the number of times the ground state was measured out of 1000 trials per each time.

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, make a plot of the raw data obtained from the experiment. Your plot should have time on the horizontal axis and probability (frequency) of measuring the ground state on the vertical axis. Axis labels and a title never hurt anyone!

In [ ]:
"""Plot the raw data."""
plt.figure(figsize=(12, 5))
plt.scatter(times, data);
plt.xlabel("Time [microseconds]");
plt.ylabel("Probability of ground state");
plt.title("Ramsey measurement data")
plt.grid();
plt.show()

### Fitting

This data looks noisy, but there is a pattern that we can fit!

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, define a function which takes as arguments $t$, $\omega$, and $T_2$. (Order matters!) Return the following functional form:

\begin{equation}
    \frac{1}{2} \left(1 +  \cos \omega t \right) e^{- t / T_2} .
\end{equation}

To reiterate, the oscillatory behavior $\cos \omega t$ is due to the qubit rotating about the $z$-axis while in the $xy$-plane. The exponential decay $e^{-t / T_2}$ is due to decoherence. 

In [ ]:
"""Define the fit function."""
def ramsey_fit(time: float, domega: float, T2: float) -> float:
    return 0.5 + 0.5 * np.cos(domega * times) * np.exp(- times / T2)

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, use `scipy.optimize.curve_fit` or your favorite data-fitting tool to fit the fit function to the raw data.

In [ ]:
"""Do the fitting."""
optimal_params, covariance = scp.optimize.curve_fit(ramsey_fit, times, data)

**Question:** What is your computed $T_2$ time? (Remember units!)

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

In [ ]:
"""Plot the fit and data."""
# Compute the fit
fitvals = ramsey_fit(times, *optimal_params)

# Do the plotting
plt.figure(figsize=(12, 5))
plt.plot(times, fitvals, "-", linewidth=4, label="Fit", color="red");
plt.scatter(times, data, label="Data");
plt.xlabel("Time [microseconds]");
plt.ylabel("Probability of ground state");
plt.title("Ramsey measurement data")
plt.legend();
plt.grid();
plt.text(45, 1.0, f"T2 = {round(optimal_params[1], 3)} microseconds")
plt.show()

## Hahn echo

Unlike energy decay, where the qubit excitation escapes into the environment, the pure dephasing of a qubit is, at least in principle, revisible. To see why this is, let's think about what dephasing actually is.

A qubit consists of two energy levels spaced by $\hbar\omega_{01}$. We often think of this energy as a fixed parameter of the system, but in reality system noise may cause $\omega_{01}$ fluctuate around, i.e. $\omega_{01} = \langle\omega_{01}\rangle + \delta\omega_{01}(t)$. A superposition of the eigenvectors of a Hamiltonian will evolve in time as

$$|\psi(t)\rangle = \alpha |{0}\rangle + e^{i\phi_0}e^{-\int_0^t\omega_{01}(t)dt}\beta |{1}\rangle = \alpha |{0}\rangle +   e^{i(\phi_0-\langle\omega_{01}\rangle t)}e^{-\int_0^t\delta\omega_{01}dt}\beta|{1}\rangle $$ 

Here, $\phi_0$ is the phase we encoded in our superposition state. The $\langle\omega_{01}\rangle t$ term represents deterministic (Shrödinger Equation) evolution of the Hamiltonian: this evolution is not a problem, as we can always correct for this as long as we know $\langle\omega_{01}\rangle$ and $t$. The second exponential captures the phase evolution caused by stochastic fluctaions in the qubit frequency. It can be shown that, as $t$ increases, this term causes the overal phase to undergo a random walk away from it's deterministic value. Since we dont know what $\delta\omega_{01}$ is, we can't correct for it: this is what causes us to lose phase information in our qubit.

However, if we were to somehow reverse time halfway through the experiment, we'd find

\begin{align}
|{\psi(t)}\rangle &= \alpha|{0}\rangle + \exp{(i\phi_0)} 
\exp{ \left[ -i\left( \int_0^{t/2}\omega_{01}(t)dt + \int_{t/2}^0\omega_{01}(t)dt \right) \right] }\beta |{1}\rangle \\
&= \alpha|{0}\rangle + \exp{ (i\phi_0)} 
\exp{\left[ -i \left(\int_0^{t/2}\omega_{01}(t)dt - \int_{0}^{t/2}\omega_{01}(t)dt \right) \right]}\beta |{1}\rangle \\
&= \alpha |{0}\rangle +   e^{i\phi_0}\beta|{1}\rangle \\
&= {\psi(t=0)}
\end{align}

Thus, if we had a way to "reverse time" at $t/2$, we could, in principle, undo the phase accumulation that causes dephasing. Of course we can't acutually reverse time... what we *can* do is apply a 'refocusing' pulse: if, at $t/2$ we rotate the qubit by $\pi$ about the $Y$, we will, in essence, mirror the qubit state on the Bloch sphere. Since the state vector is now mirrored on the Bloch sphere, the evolution forward in time will now *undo any evolution that happened during the first half of the experiment*. 

Take a look at the animation below.

In [ ]:
"""Video showing conceptual visualization of the above experiment on the Bloch sphere."""
from IPython.display import YouTubeVideo
YouTubeVideo("wkuqcCVhl04", width=640, height=360)

We start the expriment with a $\pi/2$ pulse about the $X$ axis, initializing it along the $-Y$ axis. Then we let the state vector rotate around for some time $t/2$, after which it makes some angle $\phi$ with the $-Y$ axis. When we apply our $Y_{\pi}$ pulse, the angle the statevector makes with the $-Y$ axis is now $-\phi$ and time evolution for $t/2$ will *undo this evolution!*

### Pulse sequence

## TODO: Add image of pulse sequence here!

### Raw data

In [ ]:
"""Load in the raw data."""
T2 = 80
sigma = 5.0
times = np.linspace(0, 100, 500)  # microseconds
data = 1 - np.exp(-times / T2) + np.random.randn(len(times)) / sigma**2

In [ ]:
"""Plot the raw data."""
plt.figure(figsize=(12, 5))
plt.plot(times, data);
plt.xlabel("Time [microseconds]");
plt.ylabel("Frequency of ground state");
plt.title("Hahn-echo measurement data");
plt.grid();
plt.show()

### Fitting

In [ ]:
"""Define a fit function."""
def hahn_fit(time: float, T2: float) -> float:
    return 1 - np.exp(-time / T2)  # Your code here!

In [ ]:
"""Do the fitting."""
optimal_params, covariance = scp.optimize.curve_fit(hahn_fit, times, data)

print("T2 =", round(optimal_params[0], 2), "microseconds.")

In [ ]:
"""Plot the fit function and data."""
# Compute the fit
fitvals = hahn_fit(times, *optimal_params)

# Plot the fit and data
plt.figure(figsize=(12, 5))
plt.plot(times, data, linewidth=2, label="Data");
plt.plot(times, fitvals, "-", linewidth=4, label="Fit");
plt.xlabel("Time [microseconds]");
plt.ylabel("Frequency of ground state");
plt.title("Hahn-echo measurement data");
plt.legend();
plt.grid();
plt.text(45, 0.2, f"T2 = {round(optimal_params[0], 3)} microseconds")
plt.show()

# Summary

## Bloch-Redfield model

\begin{equation}
    \rho_{BR} := \left[ \begin{matrix}
        1 + (|\alpha|^2 - 1) e^{-t / T_1} & \alpha \beta^* e^{-t / T_2} \\
        \alpha^* \beta e^{-t / T_2} & |\beta|^2 e^{- t / T_1} 
    \end{matrix} \right]
\end{equation}

In [ ]:
"""Visualize the Bloch-Redfield density matrix."""
def rho(t: float, 
        alpha0: complex = np.sqrt(1 / 2), 
        beta0: complex = np.sqrt(1 / 2),
        T1: float = 40,
        T2: float = 70) -> np.ndarray:
    return np.array([
        [1 + (abs(alpha0)**2 - 1) * np.exp(- t / T1), alpha0 * np.conj(beta0) * np.exp(-t / T2)],
        [np.conj(alpha0) * beta0 * np.exp(-t / T2), abs(beta0)**2 * np.exp(-t / T1)]
    ])

In [ ]:
"""Set your parameters here!"""
wait_time = 0.05
tmax = 200
nsteps = 100

In [ ]:
"""Run this cell to do the animation."""
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)

for t in np.linspace(0.01, tmax, nsteps):
    ax.cla()
    plt.imshow(np.real(rho(t)), 
               cmap="Greens",
               vmin=0, 
               vmax=1)
    plt.title("t = %0.2f microseconds" % round(t, 2))
    plt.colorbar()
    plt.axis("off")
    display(fig)
    clear_output(wait=True)
    plt.pause(wait_time)

In [ ]:
"""Plot the matrix elements of rho over time."""
# Initial state
alpha = np.sqrt(0.3)
beta = np.sqrt(0.7)

# T1 and T2 times
T1 = 50
T2 = 70

# Time
time = np.linspace(0, 250, 100)

# Matrix elements
rho00 = 1 + (abs(alpha)**2 - 1) * np.exp(-time / T1)
rho01 = alpha * np.conj(beta) * np.exp(-time / T2)
rho10 = np.conj(rho01)
rho11 = 1 - rho00

# Plotting
plt.figure(figsize=(16, 5));
plt.plot(time, rho00, "--", label=r"\rho_{00}", linewidth=4);
plt.plot(time, np.real(rho01), "--", label=r"\rho_{01}", linewidth=4);
plt.plot(time, np.real(rho10), "--", label=r"\rho_{10}", linewidth=4);
plt.plot(time, rho11, "--", label=r"\rho_{11}", linewidth=4);
plt.grid();
plt.legend();
plt.xlabel("Time [microseconds]");

# Further reading and resources

* [Quantum engineer's guide to superconducting qubits](https://arxiv.org/abs/1904.06560).
* [Decoherence textbook](https://www.springer.com/gp/book/9783540357735).
* [Qiskit Pulse documentation (or lack thereof)](https://qiskit.org/documentation/apidoc/pulse/pulse.html).
* [Qiskit textbook notebook on calibrating qubits](https://community.qiskit.org/textbook/ch-quantum-hardware/calibrating-qubits-openpulse.html).

# Technical notes

1. Superposition and coherence are basis-dependant. That is, the existence of coherence depends on the particular basis one is using. For example, the state $|0\rangle + |1\rangle$ is a coherent superposition of the computational basis states $|0\rangle$ and $|1\rangle$, but it is one of the $X$-basis states $|+\rangle$. So, in the computational basis, this density matrix does have off-diagonal elements, but in the $X$-basis this density matrix would be diagonal. For the discussion in the main notebook, we always use coherence to mean coherence in the computational basis. Decoherence causes a decay to some "preferred basis" (which happens to be the computational basis in superconducting qubits). 